In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, shape, Point

from osgeo import gdal
from osgeo import ogr

import pandas as pd
import gempy as gp
import gemgis as gg

# from dask_rasterio import read_raster, write_raster

import os
from sklearn.preprocessing import OrdinalEncoder

Not subsurface compatibility available


### NOTE:

In order to run this notebook you must uncomment out a section in thedocker file and make a new image. Uncommenting the following lines will cause issues with gdal. It is reccomended to use this configutation only with this file.

This is only for extracting X values. It will cause gdal to fail

RUN pip uninstall pandas

RUN pip install pandas==1.3.4

RUN pip install pyvista gempy

RUN apt-get install ffmpeg libsm6 libxext6  -y


## Shoreline
---

In [3]:
shape = '/workspace/data-sets/nyc_shoreline/NYC_2017_LiDAR_Low_Tide_Shoreline.shp'
v_df = gpd.read_file(shape)
v_df.head()

,Feature,Type,geometry
0,Shoreline,Bathymetric - low tide,"LINESTRING Z (914245.542 119924.624 0.000, 914..."
1,Shoreline,Bathymetric - low tide,"LINESTRING Z (921853.440 120210.369 0.000, 921..."
2,Shoreline,Bathymetric - low tide,"LINESTRING Z (919613.833 121459.244 0.000, 919..."
3,Shoreline,Bathymetric - low tide,"LINESTRING Z (919666.992 121515.874 0.000, 919..."
4,Shoreline,Bathymetric - low tide,"LINESTRING Z (921742.418 121910.178 0.000, 921..."


In [4]:
v_df['tag']=v_df.index
v_df.head()

,Feature,Type,geometry,tag
0,Shoreline,Bathymetric - low tide,"LINESTRING Z (914245.542 119924.624 0.000, 914...",0
1,Shoreline,Bathymetric - low tide,"LINESTRING Z (921853.440 120210.369 0.000, 921...",1
2,Shoreline,Bathymetric - low tide,"LINESTRING Z (919613.833 121459.244 0.000, 919...",2
3,Shoreline,Bathymetric - low tide,"LINESTRING Z (919666.992 121515.874 0.000, 919...",3
4,Shoreline,Bathymetric - low tide,"LINESTRING Z (921742.418 121910.178 0.000, 921...",4


In [5]:
gdf_xyz = gg.vector.extract_xyz(gdf=v_df)
gdf_xyz.head(10)

,Feature,Type,tag,geometry,X,Y,Z
0,Shoreline,Bathymetric - low tide,0,POINT (914245.542 119924.624),914245.54,119924.62,0.00
1,Shoreline,Bathymetric - low tide,0,POINT (914246.358 119924.204),914246.36,119924.20,0.00
2,Shoreline,Bathymetric - low tide,0,POINT (914247.598 119925.011),914247.60,119925.01,0.00
3,Shoreline,Bathymetric - low tide,0,POINT (914249.239 119924.992),914249.24,119924.99,0.00
4,Shoreline,Bathymetric - low tide,0,POINT (914250.870 119924.153),914250.87,119924.15,0.00
5,Shoreline,Bathymetric - low tide,0,POINT (914252.502 119923.314),914252.50,119923.31,0.00
6,Shoreline,Bathymetric - low tide,0,POINT (914253.996 119922.476),914254.00,119922.48,0.00
7,Shoreline,Bathymetric - low tide,0,POINT (914256.043 119922.043),914256.04,119922.04,0.00
8,Shoreline,Bathymetric - low tide,0,POINT (914258.778 119922.012),914258.78,119922.01,0.00
9,Shoreline,Bathymetric - low tide,0,POINT (914260.834 119922.399),914260.83,119922.40,0.00


In [6]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()
v_df['z'] = z_val['Z']
v_df.head()

,Feature,Type,geometry,tag,z
0,Shoreline,Bathymetric - low tide,"LINESTRING Z (914245.542 119924.624 0.000, 914...",0,0.00
1,Shoreline,Bathymetric - low tide,"LINESTRING Z (921853.440 120210.369 0.000, 921...",1,0.00
2,Shoreline,Bathymetric - low tide,"LINESTRING Z (919613.833 121459.244 0.000, 919...",2,0.00
3,Shoreline,Bathymetric - low tide,"LINESTRING Z (919666.992 121515.874 0.000, 919...",3,0.00
4,Shoreline,Bathymetric - low tide,"LINESTRING Z (921742.418 121910.178 0.000, 921...",4,0.00


In [7]:
v_df.to_file('/workspace/data-sets/shoreline-z/shoreline-z.shp')

## Retaining Walls
---

In [9]:
shape = '/workspace/data-sets/RETAININGWALL/RETAININGWALL.shp'
v_df = gpd.read_file(shape)
v_df['tag']=v_df.index
v_df.head()

,SOURCE_ID,FEAT_CODE,SUB_CODE,STATUS,SHAPE_Leng,geometry,tag
0,21400000001.00,4000,400000,Unchanged,365.72,"LINESTRING Z (985530.573 194711.453 44.408, 98...",0
1,21400000002.00,4000,400000,Unchanged,226.16,"LINESTRING Z (985475.003 194408.890 68.260, 98...",1
2,21400000003.00,4000,400000,Unchanged,50.95,"LINESTRING Z (985523.914 194769.822 22.721, 98...",2
3,21400000004.00,4000,400000,Unchanged,175.28,"LINESTRING Z (987357.515 194593.661 68.980, 98...",3
4,21400000005.00,4000,400000,Unchanged,100.05,"LINESTRING Z (987949.170 194143.960 70.970, 98...",4


In [10]:
gdf_xyz = gg.vector.extract_xyz(gdf=v_df)
gdf_xyz.head(10)

,SOURCE_ID,FEAT_CODE,SUB_CODE,STATUS,SHAPE_Leng,tag,geometry,X,Y,Z
0,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985530.573 194711.453),985530.57,194711.45,44.41
1,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985516.974 194716.568),985516.97,194716.57,43.78
2,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985466.945 194735.385),985466.94,194735.39,41.45
3,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985460.862 194737.673),985460.86,194737.67,41.16
4,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985460.392 194736.371),985460.39,194736.37,41.17
5,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985403.868 194579.722),985403.87,194579.72,42.35
6,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985402.487 194575.893),985402.49,194575.89,42.38
7,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985457.311 194556.506),985457.31,194556.51,42.76
8,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985469.605 194552.159),985469.60,194552.16,42.84
9,21400000001.00,4000,400000,Unchanged,365.72,0,POINT (985514.911 194536.137),985514.91,194536.14,43.15


In [11]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()
v_df['z'] = z_val['Z']
v_df.head()

,SOURCE_ID,FEAT_CODE,SUB_CODE,STATUS,SHAPE_Leng,geometry,tag,z
0,21400000001.00,4000,400000,Unchanged,365.72,"LINESTRING Z (985530.573 194711.453 44.408, 98...",0,44.41
1,21400000002.00,4000,400000,Unchanged,226.16,"LINESTRING Z (985475.003 194408.890 68.260, 98...",1,43.78
2,21400000003.00,4000,400000,Unchanged,50.95,"LINESTRING Z (985523.914 194769.822 22.721, 98...",2,41.45
3,21400000004.00,4000,400000,Unchanged,175.28,"LINESTRING Z (987357.515 194593.661 68.980, 98...",3,41.16
4,21400000005.00,4000,400000,Unchanged,100.05,"LINESTRING Z (987949.170 194143.960 70.970, 98...",4,41.17


In [12]:
v_df.to_file('/workspace/data-sets/retainingwall-z/retainingwall-z.shp')

## Transport Structure
---

In [13]:
data = '/workspace/data-sets/doitt_planimetrics.gdb'
layer_list = fiona.listlayers(data)
layer_list.remove('RETAININGWALL')
layer_list.remove('HYDROGRAPHY')
layer_list.remove('SHORELINE')
layer_list.remove('ELEVATION')

In [17]:
v_df = gpd.read_file(data, layer='TRANSPORT_STRUCTURE')
v_df['tag']=v_df.index
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,geometry,tag
0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,MULTIPOLYGON Z (((985711.962 199792.396 66.631...,0
1,None,21233000002.00,2330,233000,Unchanged,3780.02,52710.83,MULTIPOLYGON Z (((985400.536 194238.887 61.660...,1
2,None,21230000003.00,2300,230000,Unchanged,5932.11,118944.16,MULTIPOLYGON Z (((984475.052 191777.236 53.745...,2
3,None,21230000004.00,2300,230000,Unchanged,5727.65,102510.67,MULTIPOLYGON Z (((984412.295 191776.928 40.353...,3
4,None,21230000005.00,2300,230000,Unchanged,9935.86,462752.09,"MULTIPOLYGON Z (((991360.925 192937.706 0.000,...",4


In [25]:
exploded = v_df.explode()

In [26]:
exploded

,,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,tag,geometry
0,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,"POLYGON Z ((985711.962 199792.396 66.631, 9857..."
1,0,None,21233000002.00,2330,233000,Unchanged,3780.02,52710.83,1,"POLYGON Z ((985400.536 194238.887 61.660, 9853..."
2,0,None,21230000003.00,2300,230000,Unchanged,5932.11,118944.16,2,"POLYGON Z ((984475.052 191777.236 53.745, 9844..."
3,0,None,21230000004.00,2300,230000,Unchanged,5727.65,102510.67,3,"POLYGON Z ((984412.295 191776.928 40.353, 9844..."
4,0,None,21230000005.00,2300,230000,Unchanged,9935.86,462752.09,4,"POLYGON Z ((991360.925 192937.706 0.000, 99147..."
...,...,...,...,...,...,...,...,...,...,...
2216,0,None,20232000073.00,2320,232000,Unchanged,16286.31,305210.15,2216,"POLYGON Z ((1011167.622 186477.638 100.132, 10..."
2217,0,None,20232000074.00,2340,234000,Updated,211.03,2522.90,2217,"POLYGON Z ((1012171.223 179370.946 0.000, 1012..."
2218,0,None,20232000076.00,2340,234000,Updated,390.40,4704.58,2218,"POLYGON Z ((1012315.669 178718.652 0.000, 1012..."
2219,0,None,20232000078.00,2320,232000,Updated,192.08,1849.47,2219,"POLYGON Z ((1012418.119 178200.214 0.000, 1012..."


In [27]:
gdf_xyz = gg.vector.extract_xyz(gdf=exploded)
gdf_xyz.head(10)

,level_2,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,tag,geometry,X,Y,Z
0,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985711.962 199792.396),985711.96,199792.40,66.63
1,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985715.732 199778.081),985715.73,199778.08,67.99
2,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985716.601 199774.781),985716.60,199774.78,67.99
3,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985730.970 199740.888),985730.97,199740.89,73.00
4,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985744.049 199710.040),985744.05,199710.04,73.00
5,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985741.339 199708.511),985741.34,199708.51,73.58
6,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985741.709 199707.612),985741.71,199707.61,73.58
7,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985756.520 199671.693),985756.52,199671.69,73.58
8,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985758.386 199667.167),985758.39,199667.17,73.58
9,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985758.555 199666.756),985758.56,199666.76,73.58


In [28]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()

In [33]:
min_z=z_val.groupby(['tag'])[['Z']].agg('min')

In [34]:
v_df['min-z'] = min_z['Z']
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,geometry,tag,min-z
0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,MULTIPOLYGON Z (((985711.962 199792.396 66.631...,0,59.86
1,None,21233000002.00,2330,233000,Unchanged,3780.02,52710.83,MULTIPOLYGON Z (((985400.536 194238.887 61.660...,1,61.11
2,None,21230000003.00,2300,230000,Unchanged,5932.11,118944.16,MULTIPOLYGON Z (((984475.052 191777.236 53.745...,2,43.49
3,None,21230000004.00,2300,230000,Unchanged,5727.65,102510.67,MULTIPOLYGON Z (((984412.295 191776.928 40.353...,3,24.54
4,None,21230000005.00,2300,230000,Unchanged,9935.86,462752.09,"MULTIPOLYGON Z (((991360.925 192937.706 0.000,...",4,0.00


In [35]:
v_df.to_file('/workspace/data-sets/transportstructure-z/transportstructure-z.shp')

## Curb
---

In [36]:
v_df = gpd.read_file(data, layer='TRANSPORT_STRUCTURE')
v_df['tag']=v_df.index
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,geometry,tag
0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,MULTIPOLYGON Z (((985711.962 199792.396 66.631...,0
1,None,21233000002.00,2330,233000,Unchanged,3780.02,52710.83,MULTIPOLYGON Z (((985400.536 194238.887 61.660...,1
2,None,21230000003.00,2300,230000,Unchanged,5932.11,118944.16,MULTIPOLYGON Z (((984475.052 191777.236 53.745...,2
3,None,21230000004.00,2300,230000,Unchanged,5727.65,102510.67,MULTIPOLYGON Z (((984412.295 191776.928 40.353...,3
4,None,21230000005.00,2300,230000,Unchanged,9935.86,462752.09,"MULTIPOLYGON Z (((991360.925 192937.706 0.000,...",4


In [37]:
exploded = v_df.explode()
gdf_xyz = gg.vector.extract_xyz(gdf=exploded)
gdf_xyz.head(10)

,level_2,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,tag,geometry,X,Y,Z
0,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985711.962 199792.396),985711.96,199792.40,66.63
1,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985715.732 199778.081),985715.73,199778.08,67.99
2,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985716.601 199774.781),985716.60,199774.78,67.99
3,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985730.970 199740.888),985730.97,199740.89,73.00
4,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985744.049 199710.040),985744.05,199710.04,73.00
5,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985741.339 199708.511),985741.34,199708.51,73.58
6,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985741.709 199707.612),985741.71,199707.61,73.58
7,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985756.520 199671.693),985756.52,199671.69,73.58
8,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985758.386 199667.167),985758.39,199667.17,73.58
9,0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,0,POINT (985758.555 199666.756),985758.56,199666.76,73.58


In [38]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()
min_z=z_val.groupby(['tag'])[['Z']].agg('min')
v_df['min-z'] = min_z['Z']
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,SHAPE_Length,SHAPE_Area,geometry,tag,min-z
0,Manhatten Bridge,21230000001.00,2300,230000,Unchanged,13869.46,762174.64,MULTIPOLYGON Z (((985711.962 199792.396 66.631...,0,59.86
1,None,21233000002.00,2330,233000,Unchanged,3780.02,52710.83,MULTIPOLYGON Z (((985400.536 194238.887 61.660...,1,61.11
2,None,21230000003.00,2300,230000,Unchanged,5932.11,118944.16,MULTIPOLYGON Z (((984475.052 191777.236 53.745...,2,43.49
3,None,21230000004.00,2300,230000,Unchanged,5727.65,102510.67,MULTIPOLYGON Z (((984412.295 191776.928 40.353...,3,24.54
4,None,21230000005.00,2300,230000,Unchanged,9935.86,462752.09,"MULTIPOLYGON Z (((991360.925 192937.706 0.000,...",4,0.00


In [39]:
v_df.to_file('/workspace/data-sets/curb-z/curb-z.shp')

## Railroad Struct
---

In [41]:
v_df = gpd.read_file(data, layer='RAILROAD_STRUCTURE')
v_df['tag']=v_df.index
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,Shape_Length,Shape_Area,geometry,tag
0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,"MULTIPOLYGON Z (((987990.649 195805.599 8.855,...",0
1,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,MULTIPOLYGON Z (((987325.201 190215.210 32.788...,1
2,None,21248000003.00,2480,248000,Unchanged,29.70,52.28,MULTIPOLYGON Z (((988910.171 190630.820 40.387...,2
3,None,21248000004.00,2480,248000,Unchanged,22.65,31.62,MULTIPOLYGON Z (((987894.110 192892.698 42.303...,3
4,None,21247000005.00,2470,247000,Updated,155.27,264.59,MULTIPOLYGON Z (((989401.436 190476.529 36.160...,4


In [42]:
exploded = v_df.explode()
gdf_xyz = gg.vector.extract_xyz(gdf=exploded)
gdf_xyz.head(10)

,level_2,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,Shape_Length,Shape_Area,tag,geometry,X,Y,Z
0,0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,0,POINT (987990.649 195805.599),987990.65,195805.60,8.85
1,0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,0,POINT (987990.061 195799.900),987990.06,195799.90,8.95
2,0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,0,POINT (987986.229 195800.291),987986.23,195800.29,8.87
3,0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,0,POINT (987986.820 195805.990),987986.82,195805.99,8.80
4,0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,0,POINT (987990.649 195805.599),987990.65,195805.60,8.85
5,0,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,1,POINT (987325.201 190215.210),987325.20,190215.21,32.79
6,0,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,1,POINT (987322.359 190210.099),987322.36,190210.10,32.71
7,0,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,1,POINT (987318.180 190212.421),987318.18,190212.42,32.70
8,0,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,1,POINT (987321.021 190217.530),987321.02,190217.53,32.80
9,0,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,1,POINT (987325.201 190215.210),987325.20,190215.21,32.79


In [43]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()
min_z=z_val.groupby(['tag'])[['Z']].agg('min')
v_df['min-z'] = min_z['Z']
v_df.head()

,NAME,SOURCE_ID,FEATURE_CODE,SUB_FEATURE_CODE,STATUS,Shape_Length,Shape_Area,geometry,tag,min-z
0,None,21248000001.00,2480,248000,Unchanged,19.16,22.06,"MULTIPOLYGON Z (((987990.649 195805.599 8.855,...",0,8.80
1,None,21248000002.00,2480,248000,Unchanged,21.26,27.95,MULTIPOLYGON Z (((987325.201 190215.210 32.788...,1,32.70
2,None,21248000003.00,2480,248000,Unchanged,29.70,52.28,MULTIPOLYGON Z (((988910.171 190630.820 40.387...,2,40.39
3,None,21248000004.00,2480,248000,Unchanged,22.65,31.62,MULTIPOLYGON Z (((987894.110 192892.698 42.303...,3,42.30
4,None,21247000005.00,2470,247000,Updated,155.27,264.59,MULTIPOLYGON Z (((989401.436 190476.529 36.160...,4,36.12


In [44]:
v_df.to_file('/workspace/data-sets/railroad-structure-z/railroad-structure-z.shp')

## Countour
---

In [3]:
shape = '/workspace/data-sets/CONTOUR_8.5.14/CONTOUR.shp'
v_df = gpd.read_file(shape)
v_df['tag']=v_df.index
v_df.head()

,SOURCE_ID,FEATURE_CO,SUB_FEATUR,ELEVATION,Shape_Leng,geometry,tag
0,19310000001.00,3100,310000,10.00,366.77,"LINESTRING Z (979308.465 177258.151 10.000, 97...",0
1,19310000002.00,3100,310000,10.00,44.80,"LINESTRING Z (979383.060 177347.820 10.000, 97...",1
2,19310000003.00,3100,310000,10.00,383.70,"LINESTRING Z (979594.390 177556.920 10.000, 97...",2
3,19310000004.00,3100,310000,10.00,1123.62,"LINESTRING Z (978475.520 176992.500 10.000, 97...",3
4,19310000005.00,3100,310000,10.00,2466.81,"LINESTRING Z (978590.900 177596.680 10.000, 97...",4


In [4]:
gdf_xyz = gg.vector.extract_xyz(gdf=v_df)
gdf_xyz.head(10)

,SOURCE_ID,FEATURE_CO,SUB_FEATUR,ELEVATION,Shape_Leng,tag,geometry,X,Y,Z
0,19310000001.00,3100,310000,10.00,366.77,0,POINT (979308.465 177258.151),979308.46,177258.15,10.00
1,19310000001.00,3100,310000,10.00,366.77,0,POINT (979307.830 177256.220),979307.83,177256.22,10.00
2,19310000001.00,3100,310000,10.00,366.77,0,POINT (979306.040 177251.030),979306.04,177251.03,10.00
3,19310000001.00,3100,310000,10.00,366.77,0,POINT (979304.140 177245.880),979304.14,177245.88,10.00
4,19310000001.00,3100,310000,10.00,366.77,0,POINT (979302.560 177241.780),979302.56,177241.78,10.00
5,19310000001.00,3100,310000,10.00,366.77,0,POINT (979301.270 177238.590),979301.27,177238.59,10.00
6,19310000001.00,3100,310000,10.00,366.77,0,POINT (979299.360 177234.180),979299.36,177234.18,10.00
7,19310000001.00,3100,310000,10.00,366.77,0,POINT (979296.820 177228.700),979296.82,177228.70,10.00
8,19310000001.00,3100,310000,10.00,366.77,0,POINT (979294.100 177223.310),979294.10,177223.31,10.00
9,19310000001.00,3100,310000,10.00,366.77,0,POINT (979289.880 177215.630),979289.88,177215.63,10.00


In [5]:
z_val = gdf_xyz[['tag', 'Z']].drop_duplicates().reset_index()
v_df['z'] = z_val['Z']
v_df.head()

,SOURCE_ID,FEATURE_CO,SUB_FEATUR,ELEVATION,Shape_Leng,geometry,tag,z
0,19310000001.00,3100,310000,10.00,366.77,"LINESTRING Z (979308.465 177258.151 10.000, 97...",0,10.00
1,19310000002.00,3100,310000,10.00,44.80,"LINESTRING Z (979383.060 177347.820 10.000, 97...",1,10.00
2,19310000003.00,3100,310000,10.00,383.70,"LINESTRING Z (979594.390 177556.920 10.000, 97...",2,10.00
3,19310000004.00,3100,310000,10.00,1123.62,"LINESTRING Z (978475.520 176992.500 10.000, 97...",3,10.00
4,19310000005.00,3100,310000,10.00,2466.81,"LINESTRING Z (978590.900 177596.680 10.000, 97...",4,10.00


In [6]:
v_df.to_file('/workspace/data-sets/contour-z/contour-z.shp')